In [1]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from PIL import Image
from sklearn.model_selection import LeaveOneOut
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [2]:
#https://keras.io/api/applications/

# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)

#freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

In [3]:
def create_model(base_model):
    
    # add a global spatial average pooling layer
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = keras.layers.Dropout(0.1)(x)
    x = Dense(100, activation='relu')(x)
    #x = keras.layers.Dropout(0.3)(x)
    
    
    # let's add a fully-connected layer
    x = Dense(231, activation='softmax')(x)
    #x = keras.layers.Dropout(0.3)(x)
    
    # and a logistic layer
    #predictions = Dense(2, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=x)
    
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=["accuracy"])
    
    return model

In [9]:
def read_images(N, num_classes):

    #Declaring array of images 
    images = []

    #loop to import images from file directory
    for i in range (12): 
        x = np.asarray(Image.open("C:/Users/Mark Fahy/Desktop/Final Year/Final Year Project/bongard-problems-master/bongard-problems-master/p"+str(N)+"/"+str(i)+".png").convert('L'))
        x = np.array((x, x, x))
        #print(x.shape)
        images.append(x)
        
    #np.tile(images, 3)
    #Converting to numpy array
    dataset = np.array(images)
    dataset = dataset.reshape((-1, 104, 104, 3))    

    #Y data
    yvalues = (0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1)
    
    # Scale images to the [0, 1] range
    dataset = dataset.astype("float32") / 255
    

    # convert class vectors to binary class matrices
    yvalues = keras.utils.to_categorical(yvalues, num_classes)    
    
    return dataset, yvalues

In [7]:
accuracies = []
losses = []
folds = 0
problems = 0

global_x_train = []
global_x_test = []
global_y_train = []
global_y_test = []

num_classes = 231
#input_shape = (104, 104, 1)

for i in range (1, 232): 
    dataset, yvalues = read_images(i, num_classes)
    #problems = problems + 1
    #print("Problem number: ", problems)
    
    x_train, x_test, y_train, y_test = train_test_split(dataset, yvalues, test_size=0.16666)  
    y_train = np.ones_like(y_train) * (i - 1)
    y_test = np.ones_like(y_test) * (i - 1)

    
    global_x_train.extend(x_train)
    global_x_test.extend(x_test)
    global_y_train.extend(y_train)
    global_y_test.extend(y_test)
    
global_x_train = np.array(global_x_train)
global_x_test = np.array(global_x_test)
global_y_train = np.array(global_y_train)
global_y_test = np.array(global_y_test)


#convert class vectors to binary class matrices
global_y_test = keras.utils.to_categorical(global_y_test, num_classes) 
global_y_train = keras.utils.to_categorical(global_y_train, num_classes) 

print(global_y_test.shape)
print(global_y_test)

(462, 231)
[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]


In [8]:
model = create_model(base_model)   
    
model.fit(global_x_train, global_y_train, epochs = 28, validation_data = (global_x_test, global_y_test))
score = model.evaluate(global_x_test, global_y_test, verbose=0)

print('Accuracy: ', score[1])
print('Loss: ', score[0])
    

#print("Average Accuracy: ", (sum(accuracies) / len(accuracies)) )
#print("Average Loss: ", (sum(losses) / len(losses)) )

Epoch 1/28
73/73 [==============================] - 21s 238ms/step - loss: 5.4574 - accuracy: 0.0152 - val_loss: 5.3010 - val_accuracy: 0.0303
Epoch 2/28
73/73 [==============================] - 20s 273ms/step - loss: 5.0458 - accuracy: 0.0485 - val_loss: 4.9430 - val_accuracy: 0.0519
Epoch 3/28
73/73 [==============================] - 20s 277ms/step - loss: 4.3828 - accuracy: 0.1195 - val_loss: 4.6407 - val_accuracy: 0.0844
Epoch 4/28
73/73 [==============================] - 20s 278ms/step - loss: 3.7931 - accuracy: 0.1957 - val_loss: 4.3732 - val_accuracy: 0.1234
Epoch 5/28
73/73 [==============================] - 21s 282ms/step - loss: 3.2960 - accuracy: 0.2641 - val_loss: 4.3624 - val_accuracy: 0.1039
Epoch 6/28
73/73 [==============================] - 21s 288ms/step - loss: 2.9456 - accuracy: 0.3216 - val_loss: 4.2913 - val_accuracy: 0.1515
Epoch 7/28
73/73 [==============================] - 21s 292ms/step - loss: 2.6255 - accuracy: 0.3900 - val_loss: 4.3716 - val_accuracy: 0.1515

In [10]:
new_model = Model(inputs=model.input, outputs=model.layers[-2].output)

#freeze all convolutional layers
for layer in new_model.layers:
    layer.trainable = False

In [15]:
accuracies = []
t_accuracies = []
num_classes = 2
problems = 0


for i in range (1, 21): 
    dataset, yvalues = read_images(i, num_classes)    
    
    x_train, x_test, y_train, y_test = train_test_split(dataset, yvalues, test_size=0.16666)
    x_train_emb = new_model.predict(x_train)
    x_test_emb = new_model.predict(x_test)
    
    x = new_model.output
    x = Dense(2, activation='softmax')(x)

    LR_model = Model(inputs=new_model.input, outputs=x)    
    LR_model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=["accuracy"])
        
    LR_model.fit(x_train, y_train, epochs = 10)
    score = LR_model.evaluate(x_test_emb, y_test, verbose=0) 
    
    accuracies.append(score[1])
    losses.append(score[0])
    problems += 1
    print("P No: ", problems)
        
        
print(accuracies)
print(losses)

print("Average Accuracy: ", (sum(accuracies) / len(accuracies)) )
print("Average Loss: ", (sum(losses) / len(losses)) )

Epoch 1/10
1/1 [==============================] - 3s 3s/step - loss: 2.0756 - accuracy: 0.3000
Epoch 2/10
1/1 [==============================] - 0s 73ms/step - loss: 1.7022 - accuracy: 0.4000
Epoch 3/10
1/1 [==============================] - 0s 73ms/step - loss: 1.2250 - accuracy: 0.3000
Epoch 4/10
1/1 [==============================] - 0s 73ms/step - loss: 1.2328 - accuracy: 0.4000
Epoch 5/10
1/1 [==============================] - 0s 74ms/step - loss: 1.0916 - accuracy: 0.4000
Epoch 6/10
1/1 [==============================] - 0s 75ms/step - loss: 0.9886 - accuracy: 0.4000
Epoch 7/10
1/1 [==============================] - 0s 78ms/step - loss: 0.4587 - accuracy: 0.8000
Epoch 8/10
1/1 [==============================] - 0s 75ms/step - loss: 0.5205 - accuracy: 0.7000
Epoch 9/10
1/1 [==============================] - 0s 74ms/step - loss: 0.7484 - accuracy: 0.4000
Epoch 10/10
1/1 [==============================] - 0s 77ms/step - loss: 0.4544 - accuracy: 0.8000


ValueError: in user code:

    File "C:\PythonEnvs\KerasEg\lib\site-packages\keras\engine\training.py", line 1366, in test_function  *
        return step_function(self, iterator)
    File "C:\PythonEnvs\KerasEg\lib\site-packages\keras\engine\training.py", line 1356, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\PythonEnvs\KerasEg\lib\site-packages\keras\engine\training.py", line 1349, in run_step  **
        outputs = model.test_step(data)
    File "C:\PythonEnvs\KerasEg\lib\site-packages\keras\engine\training.py", line 1303, in test_step
        y_pred = self(x, training=False)
    File "C:\PythonEnvs\KerasEg\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\PythonEnvs\KerasEg\lib\site-packages\keras\engine\input_spec.py", line 227, in assert_input_compatibility
        raise ValueError(f'Input {input_index} of layer "{layer_name}" 